In [1]:
#CSE 6363 - Machine Learning
#Team 13 - Project Submission 2

    #Shruthi Sree Thirunavukkarasu (1001933428)
    #Dinesh Kumar KodithiPalli Lava Kumar (1001949238)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to C:\Users\shruthi
[nltk_data]     sree\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\shruthi
[nltk_data]     sree\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\shruthi
[nltk_data]     sree\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

#Reading input file with dataset
rawData=pd.read_json(r"/content/drive/MyDrive/Colab Notebooks/AMAZON_FASHION.json.gz",lines=True,orient='columns')
rawData.head()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#Removing duplicate reviews
print("The shape of the data set before removing duplicate reviews : {}".format(rawData.shape))
rawData=rawData.drop_duplicates(subset=["reviewText"], keep='first', inplace=False)
print("The shape of the data set after removing duplicate reviews : {}".format(rawData.shape))

In [ ]:
#Extracting required columns from the dataset
reviewData = rawData[["reviewerID","reviewText","overall","asin"]]
reviewData = reviewData.sample(n=40000)
reviewData.info()

In [ ]:
#Removing null entries from the dataset
uniqueData = reviewData.dropna()
uniqueData.info()
uniqueData.head()

In [ ]:
#Generating Wordcloud for the reviews
reviewTextData = uniqueData['reviewText']
wordcloud = WordCloud(background_color='white',
                      width=1000,
                      height=1000
                     ).generate(" ".join(reviewTextData))
plt.figure(figsize=(10,5))
plt.imshow(wordcloud)
plt.title('All Words in the Reviews\n',size=20)
plt.axis('off')
plt.show()

In [ ]:
#Joining all review texts into a single text line separated by space
reviewTextData = " ".join(uniqueData['reviewText'])

#Creating list of sentiment words to compare and measure frequency
words = ['awesome','great','fantastic','extraordinary','amazing','super',
                 'magnificent','stunning','impressive','wonderful','breathtaking',
                 'love','content','pleased','happy','glad','satisfied','lucky',
                 'shocking','cheerful','wow','sad','unhappy','horrible','regret',
                 'bad','terrible','annoyed','disappointed','upset','awful','hate']


#Counting frequency of sentiment words in review text
wordFrequency = {}
for word in reviewTextData.split(" "):
    if word in words:
        wordFrequency[word] = wordFrequency.get(word,0)+1

In [ ]:
#Generating wordcloud of the most frequent sentiment words
wordcloud = WordCloud(background_color='white',
                      width=1000,
                      height=400
                     ).generate_from_frequencies(wordFrequency)
plt.figure(figsize=(10,5))
plt.imshow(wordcloud)
plt.title('Sentiment Words\n',size=20)
plt.axis('off')
plt.show()

In [ ]:
#Plotting and counting the distribution of reviews based on ratings between 1-5
plt.figure(figsize=(10,5))
sns.countplot(uniqueData['overall'])
plt.title('Count ratings')
plt.show()

In [ ]:
#Removing punctuations from the review texts
remPunct = re.compile("[.;:*~!\'?,\"()\\\[\]]") 
remPunctWSpc = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
uniqueData.reviewText = [remPunct.sub("", i.lower()) for i in uniqueData.reviewText] 
uniqueData.reviewText= [remPunctWSpc.sub(" ", i) for i in uniqueData.reviewText]
print(uniqueData)

In [ ]:
#Removing stop words from the 
english_stopwords = stopwords.words('english')
print(english_stopwords,"\n")
def remove_stopwords(reviewCollection):
    removed_stopwords = []
    for reviews in reviewCollection:
        removed_stopwords.append(
            ' '.join([word for word in reviews.split() 
                      if word not in english_stopwords])
        )
    return removed_stopwords
without_stopwords = remove_stopwords(uniqueData.reviewText)

print(without_stopwords[:5])

In [ ]:
#Stemming the refined review texts
def get_stemmed_text(reviewCollection):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in reviews.split()]) for reviews in reviewCollection]

stemmed_reviews = get_stemmed_text(without_stopwords)

print(stemmed_reviews[:5])

In [ ]:
#Lemmatizing the refined review texts
def get_lemmatized_text(reviewCollection):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in reviews.split()]) for reviews in reviewCollection]

lemmatized_reviews = get_lemmatized_text(without_stopwords)
print(lemmatized_reviews[:5])


In [ ]:
#Data Labelling steps
uniqueData['reviewText'] = lemmatized_reviews
conditions = [
    (uniqueData['overall'] >= 4),
    (uniqueData['overall'] == 3),
    (uniqueData['overall'] <= 2)
    ]
values = ['Positive', 'Neutral', 'Negative']
uniqueData['sentiment'] = np.select(conditions, values)
uniqueData.head()

In [ ]:
#Data Vectorization steps
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
corpus = ['exactly needed', 'agree review opening small almost bent hook expensive earring trying get higher end theyre seen would buy price sending back', 'love going order another pack keep work someone including always losing back earring dont understand fish hook earring dont wish tiny bit longer', 'tiny opening', 'okay']
vectors = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
print(f"feature names\n{feature_names}")
matrix = vectors.todense()
list_dense = matrix.tolist()
dataFrame= pd.DataFrame(list_dense, columns=feature_names)
print(dataFrame)

In [ ]:
#Splitting data into test and train sets
x = uniqueData['reviewText'].values
y = uniqueData['sentiment'].values
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 0,stratify = y)

In [ ]:
np.unique(y_train,return_counts=True)

In [ ]:
np.unique(y_test,return_counts=True)

In [ ]:
#SVM
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())])
text_model.fit(x_train,y_train)
y_pred = text_model.predict(x_test)
np.unique(y_pred,return_counts=True)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_pred,y_test)*100
print(classification_report(y_train, y_pred, target_names=target_names))

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))